<a href="https://colab.research.google.com/github/romicabrera/Data-science/blob/main/skincare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Carga y exploración de datos
spark = SparkSession.builder.appName("SkinCareML").getOrCreate()
df = spark.read.csv("skincare_productos.csv", header=True, inferSchema=True)
df.show()
df.describe(["SPF"]).show()

# 2. Preprocesamiento
# Mapear variables categóricas a numéricas
hidratacion_map = {"Bajo": 0, "Medio": 1, "Alto": 2}
absorcion_map = {"Bajo": 0, "Medio": 1, "Alto": 2}
tipo_piel_map = {"Seco": 0, "Graso": 1, "Mixto": 2, "Sensible": 3}
df = df.withColumn("Hidratacion_num", when(col("Hidratacion") == "Bajo", 0).when(col("Hidratacion") == "Medio", 1).when(col("Hidratacion") == "Alto", 2))
df = df.withColumn("Absorcion_num", when(col("Absorcion") == "Bajo", 0).when(col("Absorcion") == "Medio", 1).when(col("Absorcion") == "Alto", 2))
df = df.withColumn("TipoPiel_num", when(col("Tipo_de_Piel") == "Seco", 0).when(col("Tipo_de_Piel") == "Graso", 1).when(col("Tipo_de_Piel") == "Mixto", 2).when(col("Tipo_de_Piel") == "Sensible", 3))

# VectorAssembler para features
assembler = VectorAssembler(inputCols=["Hidratacion_num", "Absorcion_num", "SPF"], outputCol="features")
df = assembler.transform(df)

# 3. División de datos y entrenamiento
train, test = df.randomSplit([0.8, 0.2], seed=42)
classifier = DecisionTreeClassifier(labelCol="TipoPiel_num", featuresCol="features")
model = classifier.fit(train)

# 4. Predicción y evaluación
predictions = model.transform(test)
predictions.select("Ingredientes", "Tipo_de_Piel", "prediction").show()
evaluator = MulticlassClassificationEvaluator(labelCol="TipoPiel_num", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo: {accuracy:.2f}")

# 5. Análisis de resultados y mejoras
print("""
El modelo de árbol de decisión alcanzó una precisión que depende de la cantidad de datos y su diversidad. Para mejorar la precisión se pueden probar algoritmos como Random Forest, ajustar hiperparámetros, o incluir más variables (por ejemplo, usando ingredientes codificados con OneHotEncoder).
""")

spark.stop()


+--------------------+-----------+---------+---+------------+
|        Ingredientes|Hidratacion|Absorcion|SPF|Tipo_de_Piel|
+--------------------+-----------+---------+---+------------+
|   Ácido Hialurónico|       Alto|    Medio|  0|        Seco|
|             Retinol|       Bajo|     Alto|  0|       Graso|
|          Vitamina C|      Medio|    Medio| 30|       Mixto|
|           Aloe Vera|       Alto|     Bajo| 15|    Sensible|
|         Niacinamida|      Medio|    Medio|  0|       Mixto|
|           Ceramidas|       Alto|     Bajo|  0|        Seco|
|    Ácido Salicílico|       Bajo|     Alto|  0|       Graso|
|   Centella Asiática|      Medio|    Medio| 20|    Sensible|
|Extracto de Té Verde|      Medio|     Alto|  0|       Mixto|
|   Manteca de Karité|       Alto|     Bajo|  0|        Seco|
| Extracto de Regaliz|      Medio|    Medio| 15|    Sensible|
|          Vitamina E|       Alto|    Medio| 25|        Seco|
|           Bakuchiol|       Bajo|    Medio|  0|       Mixto|
|       